In [1]:
import os
from gamma.simulator.gamma import domain_mgr, heat_solve_mgr
import cupy as cp
import numpy as np
import pyvista as pv
import vtk
import zarr as z

# Before starting, check using nvidia-smi
cp.cuda.Device(0).use()

<CUDA Device 0>

In [4]:
def save_vtk(filename):
    active_elements = domain.elements[domain.active_elements].tolist()
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = domain.nodes.get()
    active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    active_grid.point_data['temp'] = heat_solver.temperature.get()
    active_grid.save(filename)

In [5]:
domain = domain_mgr(filename='wall.k')
heat_solver = heat_solve_mgr(domain)

Time of reading input files: 1.264573335647583
Time of calculating critical timestep: 0.32566165924072266
Time of reading and interpolating toolpath: 0.03569817543029785
Number of nodes: 96874
Number of elements: 83270
Number of time-steps: 142500
Time of generating surface: 1.572815179824829


In [6]:
root_group = z.group('example.zarr')
g1 = root_group.create_group('g1')
g2 = g1.create_group('g2')
d1 = g2.create_dataset('temperature', shape=(1, domain.nodes.shape[0]))

In [10]:
output_step = 0.5  # output time step
# initialization
file_num = 0
if not os.path.exists('./vtk'):
    os.mkdir('./vtk')

# save file
filename = 'vtk/u{:05d}.vtk'.format(file_num)
save_vtk(filename)
file_num = file_num + 1
output_time = domain.current_sim_time
# time loop
while domain.current_sim_time < domain.end_sim_time:
    # here is where you want to make changes at each time step
    heat_solver.time_integration()

    d1.append(cp.expand_dims(heat_solver.temperature, axis=0).get(), axis=0)
    # save file
    if domain.current_sim_time >= output_time + output_step:
        print("Current time:  {}, Percentage done:  {}%".format(
            domain.current_sim_time, 100 * domain.current_sim_time / domain.end_sim_time))
        filename = 'vtk/u{:05d}.vtk'.format(file_num)
        save_vtk(filename)
        file_num = file_num + 1
        output_time = domain.current_sim_time

Current time:  1.0000000000000007, Percentage done:  0.3508771929824564%
Current time:  1.500000000000001, Percentage done:  0.5263157894736846%
Current time:  2.0000000000000013, Percentage done:  0.7017543859649128%


These are the parameters you might need for this project:

In [9]:
# absorpted laser power
domain.dt

0.002

In [6]:
# nodal temperature array
heat_solver.temperature

array([300.03985522, 367.72030422, 365.4099788 , ..., 300.        ,
       300.        , 300.        ])

In [7]:
# node coordinates
domain.nodes

array([[ 22.626017,   0.892857,  -8.      ],
       [  3.533256,  -1.680824,  -2.      ],
       [  3.155609,  -1.686837,  -2.      ],
       ...,
       [-23.      ,   1.25    ,  30.      ],
       [-23.      ,  -1.25    ,  29.625   ],
       [-23.      ,  -1.25    ,  30.      ]])